In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
HMC_path = Path("../files/HMC.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
hmc_df = pd.read_csv(HMC_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([hmc_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,32.980000,33.340000,32.910000,33.299999,27.475441,850000.0,13.41,188.309998,0.09,33.549999
2014-05-01,33.560001,33.700001,33.470001,33.669998,27.780720,808400.0,13.25,188.330002,0.09,33.049999
2014-05-02,33.950001,34.060001,33.680000,33.720001,27.821978,773400.0,12.91,188.059998,0.09,32.930000
2014-05-05,33.770000,33.840000,33.590000,33.799999,27.887989,770900.0,13.29,188.419998,0.09,32.880001
2014-05-06,33.740002,33.820000,33.610001,33.660000,27.772470,395000.0,13.80,186.779999,0.09,33.770000
2014-05-07,33.330002,33.590000,33.169998,33.549999,27.681711,1076400.0,13.40,187.880005,0.09,33.480000
2014-05-08,33.049999,33.250000,33.009998,33.049999,27.269161,684000.0,13.43,187.679993,0.09,33.060001
2014-05-09,32.950001,33.009998,32.799999,32.930000,27.170158,1087700.0,12.92,187.960007,0.09,33.349998
2014-05-12,32.689999,32.919998,32.680000,32.880001,27.128910,782400.0,12.23,189.789993,0.09,33.630001
2014-05-13,33.340000,33.840000,33.340000,33.770000,27.863232,1130100.0,12.13,189.960007,0.09,33.610001


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,36.560001,36.669998,36.360001,36.430000,36.430000,2260600.0,14.33,512.859985,5.33,37.279999,35.540,63.892458,36.526328,35.540,34.553672
2024-03-19,36.799999,37.060001,36.779999,36.849998,36.849998,1371900.0,13.82,515.710022,5.33,37.279999,35.651,66.799419,36.703574,35.651,34.598426
2024-03-20,36.849998,37.160000,36.849998,37.080002,37.080002,3765800.0,13.04,520.479980,5.33,37.320000,35.758,68.304340,36.934088,35.758,34.581912
2024-03-21,37.290001,37.490002,37.200001,37.439999,37.439999,1361200.0,12.92,522.200012,5.33,37.230000,35.843,70.554124,37.238815,35.843,34.447185
2024-03-22,37.660000,37.900002,37.599998,37.680000,37.680000,1312500.0,13.06,521.210022,5.33,36.830002,35.936,71.981958,37.555323,35.936,34.316677


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,850000.0,13.41,188.309998,0.09,33.549999,NaN,NaN,NaN,NaN,NaN
2014-05-01,808400.0,13.25,188.330002,0.09,33.049999,NaN,100.000000,NaN,NaN,NaN
2014-05-02,773400.0,12.91,188.059998,0.09,32.930000,NaN,100.000000,NaN,NaN,NaN
2014-05-05,770900.0,13.29,188.419998,0.09,32.880001,NaN,100.000000,NaN,NaN,NaN
2014-05-06,395000.0,13.80,186.779999,0.09,33.770000,NaN,74.712972,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/HMC_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,446700.0,11.68,191.380005,0.09,34.900002,33.6925,65.723698,34.823328,33.6925,32.561671
2014-05-29,1198000.0,11.57,192.369995,0.09,35.029999,33.7710,68.769083,35.000751,33.7710,32.541248
2014-05-30,526200.0,11.40,192.679993,0.09,35.090000,33.8465,71.860433,35.226389,33.8465,32.466610
2014-06-02,428400.0,11.58,192.899994,0.10,35.000000,33.9240,72.705144,35.441246,33.9240,32.406753
2014-06-03,400700.0,11.87,192.800003,0.10,34.849998,33.9930,70.428353,35.608817,33.9930,32.377182


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
Average R-squared (Train): -88.50120704146943
Average R-squared (Test): -90.25994832225055


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
Model 1 - Train R-squared: 0.18472868423862965, Test R-squared: 0.18654402855299612, Train MAE: 2.7671192255033685, Train MSE: 8.11485275350746, Test MAE: 2.8319076535656618, Test MSE: 8.445582660988059
Model 2 - Train R-squared: 0.09655529479632385, Test R-squared: 0.18512978748757747, Train MAE: 2.884837529928491, Train MSE: 8.944579930037198, Test MAE: 2.8504915633871244, Test MSE: 8.80520890759716
Model 3 - Train R-squared: 0.13930117750344695, Test R-squared: 0.1625426052642146, Train MAE: 2.7835894303960327, Train MSE: 8.602604344736527, Test MAE: 2.725737470764765, Test MSE: 8.353241077830562
Model 4 - Train R-squared: -0.9216203745927365, Test R-squared: -1.2780072143876127, Train MAE: 4.349376986152556, Train MSE: 19.540741139545734, Test MAE: 4.2571158441222545, Test MSE: 18.75696975188981
Model 5 - Train R-squared: 0.15535758018808965, Test R-squared: 0.23304685003991377, Train MAE: 2.79655068925

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.18472868423862965, Test R-squared: 0.18654402855299612, Train MAE: 2.7671192255033685, Train MSE: 8.11485275350746, Test MAE: 2.8319076535656618, Test MSE: 8.445582660988059


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


,Actual,Predicted
date,,
2014-05-28,34.599998,26.650166
2014-05-29,34.869999,26.852955
2014-05-30,35.180000,27.055485
2014-06-02,35.270000,27.144571
2014-06-03,35.180000,27.047764
...,...,...
2024-03-18,36.430000,27.811874
2024-03-19,36.849998,28.066883
2024-03-20,37.080002,28.520029


In [22]:
predictions_df.to_csv('../predicted_data/HMC_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
               Actual  Predicted
date                            
2014-05-28  34.599998  26.650166
2014-05-29  34.869999  26.852955
2014-05-30  35.180000  27.055485
2014-06-02  35.270000  27.144571
2014-06-03  35.180000  27.047764
...               ...        ...
2024-03-18  36.430000  27.811874
2024-03-19  36.849998  28.066883
2024-03-20  37.080002  28.520029
2024-03-21  37.439999  28.539886
2024-03-22  37.680000  28.940372

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,34.599998,26.650166,-22.976396
2014-05-29,34.869999,26.852955,-22.991237
2014-05-30,35.180000,27.055485,-23.094131
2014-06-02,35.270000,27.144571,-23.037790
2014-06-03,35.180000,27.047764,-23.116078
...,...,...,...
2024-03-18,36.430000,27.811874,-23.656672
2024-03-19,36.849998,28.066883,-23.834777
2024-03-20,37.080002,28.520029,-23.085147


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 23.978586888064854
